# 사용 패키지

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import lightgbm as lgb

# 데이터 로드

In [2]:
total = pd.read_csv('한국가스공사_시간별 공급량_20181231.csv', encoding='cp949')

In [3]:
total.head()

,연월일,시간,구분,공급량
0,2013-01-01,1,A,2497.129
1,2013-01-01,2,A,2363.265
2,2013-01-01,3,A,2258.505
3,2013-01-01,4,A,2243.969
4,2013-01-01,5,A,2344.105


# 전처리

In [4]:
total['구분']

0         A
1         A
2         A
3         A
4         A
         ..
368083    H
368084    H
368085    H
368086    H
368087    H
Name: 구분, Length: 368088, dtype: object

In [5]:
total['구분'].unique()

array(['A', 'B', 'C', 'D', 'E', 'G', 'H'], dtype=object)

In [6]:
# 구분 A~H string을 0~6 integar로 바꿔주기
d_map = {}
for i, d in enumerate(total['구분'].unique()):
    d_map[d] = i
total['구분'] = total['구분'].map(d_map)

In [7]:
total['구분']

0         0
1         0
2         0
3         0
4         0
         ..
368083    6
368084    6
368085    6
368086    6
368087    6
Name: 구분, Length: 368088, dtype: int64

In [8]:
# datatype를 object에서 datetime으로 바궈줌
total['연월일'] = pd.to_datetime(total['연월일'])

In [9]:
total

,연월일,시간,구분,공급량
0,2013-01-01,1,0,2497.129
1,2013-01-01,2,0,2363.265
2,2013-01-01,3,0,2258.505
3,2013-01-01,4,0,2243.969
4,2013-01-01,5,0,2344.105
...,...,...,...,...
368083,2018-12-31,20,6,681.033
368084,2018-12-31,21,6,669.961
368085,2018-12-31,22,6,657.941
368086,2018-12-31,23,6,610.953


In [10]:
#연/월/일/weekday(월화수목금토일을 숫자로) 분리
total['year'] = total['연월일'].dt.year
total['month'] = total['연월일'].dt.month
total['day'] = total['연월일'].dt.day
total['weekday'] = total['연월일'].dt.weekday

In [119]:
total.groupby([])

In [12]:
train_years = [2013,2014,2015,2016,2017]
val_years = [2018]

In [13]:
train = total[total['year'].isin(train_years)]
val = total[total['year'].isin(val_years)]

In [14]:
features = ['구분', 'month', 'day', 'weekday', '시간']
train_x = train[features]
train_y = train['공급량']

val_x = val[features]
val_y = val['공급량']

In [101]:
train_x

,구분,month,day,weekday,시간
0,0,1,1,1,1
1,0,1,1,1,2
2,0,1,1,1,3
3,0,1,1,1,4
4,0,1,1,1,5
...,...,...,...,...,...
306763,6,12,31,6,20
306764,6,12,31,6,21
306765,6,12,31,6,22
306766,6,12,31,6,23


In [16]:
train_y

0         2497.129
1         2363.265
2         2258.505
3         2243.969
4         2344.105
            ...   
306763     517.264
306764     530.896
306765     506.287
306766     470.638
306767     444.618
Name: 공급량, Length: 306768, dtype: float64

# 학습

### LGBM 주요 파라미터 
boosting : 어떤 알고리즘으로 학습할지 선택

num_boost_round : epoch 수라고 생각하면 될듯

learning_rate : 0.001 / 0.005/ 0.01등 설정 -->ANN처럼 adam optimizer가 없어서 고정값인가?? 아니 optimizer가 없내

metric : mae로 학습할건지 rmse로 학습할건지 mse로 학습할건지 기준 정해주는것

max_depth : 트리가 어디까지 뻗어나갈건지 정해주는 변수

num_leaves : 복잡성을 담당하는 parameter, 2^(max_depth)보다 작아야함.

정확도 <-------------> 과적합

큰 max_bin<----> 작은 max_bin

큰 num_leaves <------>작은 num_leaves

작은 learning rate(정확도)

큰 num_boost_round(정확도)

boosting알고리즘으로 dart 사용(정확도)

## grid search

In [37]:
boosting = ['gbdt', 'dart', 'goss']
learning_rate = [0.01,0.05,0.1,0.5,1]
max_depth = [5, 10, 15]
num_leaves = [5, 10, 15]


In [41]:
num_of_case = len(boosting)*len(learning_rate)*len(max_depth)*len(num_leaves)
num_of_case

135

In [38]:
len(boosting)

3

In [49]:
MAE_grid_search = pd.DataFrame(np.zeros((num_of_case,6)),columns = ['case number','boosting','Lr','max_depth','num_leaves','MAE'])

In [54]:
d_train = lgb.Dataset(train_x, train_y)
d_val = lgb.Dataset(val_x, val_y)
cnt = 0
for i in range (len(boosting)):
    boosting_par = boosting[i]
    
    for j in range(len(learning_rate)):
        Lr_par = learning_rate[j]
        
        for k in range(len(max_depth)):
            max_depth_par = max_depth[k]
            
            for l in range(len(num_leaves)):
                num_leaves_par = num_leaves[l]
                
                print('-'*60)
                print('Case : ',cnt+1)
                print('boosting parameter : ',boosting_par)
                print('Learning_rate : ',Lr_par)
                print('max_depth : ',max_depth_par)
                print('num_leaves : ',num_leaves_par)

                params = {
                    'objective': 'regression',
                    'metric':'mae',
                    'seed':42 ,
                    'boosting' : boosting_par,
                    'learning_rate' : Lr_par,
                    'max_depth' : max_depth_par,
                    'num_leaves' : num_leaves_par
                    
                }

                model = lgb.train(params, d_train, 1000, d_val, verbose_eval=20, early_stopping_rounds=10)
#                 model.best_score['valid_0']['l1']
                
                MAE_grid_search.iloc[cnt,0] = int(cnt+1)
                MAE_grid_search.iloc[cnt,1] = boosting_par
                MAE_grid_search.iloc[cnt,2] = Lr_par
                MAE_grid_search.iloc[cnt,3] = max_depth_par
                MAE_grid_search.iloc[cnt,4] = num_leaves_par
                MAE_grid_search.iloc[cnt,5] = model.best_score['valid_0']['l1']
                
                cnt = cnt+1


------------------------------------------------------------
Case :  1
boosting parameter :  gbdt
Learning_rate :  0.001
max_depth :  5
num_leaves :  5
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001883 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 84
[LightGBM] [Info] Number of data points in the train set: 306768, number of used features: 5
[LightGBM] [Info] Start training from score 934.864036
Training until validation scores don't improve for 10 rounds
[20]	valid_0's l1: 751.326
[40]	valid_0's l1: 745.362
[60]	valid_0's l1: 739.581
[80]	valid_0's l1: 734.444
[100]	valid_0's l1: 729.391
[120]	valid_0's l1: 724.519
[140]	valid_0's l1: 719.732
[160]	valid_0's l1: 714.687
[180]	valid_0's l1: 709.458
[200]	valid_0's l1: 704.244
[220]	valid_0's l1: 699.27
[240]	valid_0's l1: 694.356
[260]	valid_0's l1: 689.449
[280]	valid_0's l1:

[980]	valid_0's l1: 540.34
[1000]	valid_0's l1: 537.057
Did not meet early stopping. Best iteration is:
[1000]	valid_0's l1: 537.057
------------------------------------------------------------
Case :  5
boosting parameter :  gbdt
Learning_rate :  0.001
max_depth :  10
num_leaves :  10
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001372 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 84
[LightGBM] [Info] Number of data points in the train set: 306768, number of used features: 5
[LightGBM] [Info] Start training from score 934.864036
Training until validation scores don't improve for 10 rounds
[20]	valid_0's l1: 748.114
[40]	valid_0's l1: 738.909
[60]	valid_0's l1: 729.908
[80]	valid_0's l1: 721.932
[100]	valid_0's l1: 714.161
[120]	valid_0's l1: 706.591
[140]	valid_0's l1: 699.022
[160]	valid_0's l1: 690.881
[180]	valid_0's l1: 682

[920]	valid_0's l1: 474.073
[940]	valid_0's l1: 470.182
[960]	valid_0's l1: 466.388
[980]	valid_0's l1: 462.619
[1000]	valid_0's l1: 458.785
Did not meet early stopping. Best iteration is:
[1000]	valid_0's l1: 458.785
------------------------------------------------------------
Case :  9
boosting parameter :  gbdt
Learning_rate :  0.001
max_depth :  15
num_leaves :  15
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001377 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 84
[LightGBM] [Info] Number of data points in the train set: 306768, number of used features: 5
[LightGBM] [Info] Start training from score 934.864036
Training until validation scores don't improve for 10 rounds
[20]	valid_0's l1: 745.512
[40]	valid_0's l1: 733.872
[60]	valid_0's l1: 722.543
[80]	valid_0's l1: 711.519
[100]	valid_0's l1: 700.63
[120]	valid_0's l1: 690

[880]	valid_0's l1: 199.121
[900]	valid_0's l1: 197.254
[920]	valid_0's l1: 195.537
[940]	valid_0's l1: 193.874
[960]	valid_0's l1: 192.234
[980]	valid_0's l1: 190.747
[1000]	valid_0's l1: 189.321
Did not meet early stopping. Best iteration is:
[1000]	valid_0's l1: 189.321
------------------------------------------------------------
Case :  13
boosting parameter :  gbdt
Learning_rate :  0.005
max_depth :  10
num_leaves :  5
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001317 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 84
[LightGBM] [Info] Number of data points in the train set: 306768, number of used features: 5
[LightGBM] [Info] Start training from score 934.864036
Training until validation scores don't improve for 10 rounds
[20]	valid_0's l1: 729.494
[40]	valid_0's l1: 704.273
[60]	valid_0's l1: 680.133
[80]	valid_0's l1: 65

[780]	valid_0's l1: 291.34
[800]	valid_0's l1: 288.098
[820]	valid_0's l1: 285.145
[840]	valid_0's l1: 282.127
[860]	valid_0's l1: 279.299
[880]	valid_0's l1: 276.533
[900]	valid_0's l1: 273.804
[920]	valid_0's l1: 271.202
[940]	valid_0's l1: 268.952
[960]	valid_0's l1: 266.856
[980]	valid_0's l1: 264.814
[1000]	valid_0's l1: 262.899
Did not meet early stopping. Best iteration is:
[1000]	valid_0's l1: 262.899
------------------------------------------------------------
Case :  17
boosting parameter :  gbdt
Learning_rate :  0.005
max_depth :  15
num_leaves :  10
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001738 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 84
[LightGBM] [Info] Number of data points in the train set: 306768, number of used features: 5
[LightGBM] [Info] Start training from score 934.864036
Training until validati

...

[20]	valid_0's l1: 704.189
[40]	valid_0's l1: 658.198
[60]	valid_0's l1: 612.808
[80]	valid_0's l1: 572.365
[100]	valid_0's l1: 536.343
[120]	valid_0's l1: 505.551
[140]	valid_0's l1: 475.763
[160]	valid_0's l1: 450.521
[180]	valid_0's l1: 426.605
[200]	valid_0's l1: 405.926
[220]	valid_0's l1: 387.525
[240]	valid_0's l1: 371.84
[260]	valid_0's l1: 357.482
[280]	valid_0's l1: 344.901
[300]	valid_0's l1: 333.513
[320]	valid_0's l1: 322.068
[340]	valid_0's l1: 311.104
[360]	valid_0's l1: 302.283
[380]	valid_0's l1: 293.804
[400]	valid_0's l1: 286.232
[420]	valid_0's l1: 280.056
[440]	valid_0's l1: 274.058
[460]	valid_0's l1: 269.197
[480]	valid_0's l1: 264.912
[500]	valid_0's l1: 260.71
[520]	valid_0's l1: 257.335
[540]	valid_0's l1: 253.986
[560]	valid_0's l1: 251.349
[580]	valid_0's l1: 248.266
[600]	valid_0's l1: 245.628
[620]	valid_0's l1: 243.047
[640]	valid_0's l1: 240.791
[660]	valid_0's l1: 238.773
[680]	valid_0's l1: 236.719
[700]	valid_0's l1: 235.22
[720]	valid_0's l1: 233.687

[40]	valid_0's l1: 323.898
[60]	valid_0's l1: 261.414
[80]	valid_0's l1: 224.637
[100]	valid_0's l1: 201.539
[120]	valid_0's l1: 189.398
[140]	valid_0's l1: 184.085
[160]	valid_0's l1: 180.18
[180]	valid_0's l1: 176.858
[200]	valid_0's l1: 175.129
[220]	valid_0's l1: 173.332
[240]	valid_0's l1: 172.379
[260]	valid_0's l1: 171.44
[280]	valid_0's l1: 170.436
[300]	valid_0's l1: 169.576
[320]	valid_0's l1: 168.535
[340]	valid_0's l1: 167.449
[360]	valid_0's l1: 166.475
[380]	valid_0's l1: 165.575
[400]	valid_0's l1: 164.853
[420]	valid_0's l1: 164.324
[440]	valid_0's l1: 163.372
[460]	valid_0's l1: 162.785
[480]	valid_0's l1: 162.12
[500]	valid_0's l1: 161.558
[520]	valid_0's l1: 160.934
[540]	valid_0's l1: 160.211
[560]	valid_0's l1: 159.814
[580]	valid_0's l1: 159.375
[600]	valid_0's l1: 159.14
[620]	valid_0's l1: 158.585
[640]	valid_0's l1: 158.147
[660]	valid_0's l1: 157.736
[680]	valid_0's l1: 157.306
[700]	valid_0's l1: 156.854
[720]	valid_0's l1: 156.58
[740]	valid_0's l1: 156.19
[

[460]	valid_0's l1: 150.199
[480]	valid_0's l1: 149.81
Early stopping, best iteration is:
[483]	valid_0's l1: 149.697
------------------------------------------------------------
Case :  124
boosting parameter :  goss
Learning_rate :  0.05
max_depth :  15
num_leaves :  5
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001575 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 84
[LightGBM] [Info] Number of data points in the train set: 306768, number of used features: 5
[LightGBM] [Info] Using GOSS
[LightGBM] [Info] Start training from score 934.864036
Training until validation scores don't improve for 10 rounds
[20]	valid_0's l1: 529.954
[40]	valid_0's l1: 403.681
[60]	valid_0's l1: 329.266
[80]	valid_0's l1: 283.738
[100]	valid_0's l1: 258.515
[120]	valid_0's l1: 243.786
[140]	valid_0's l1: 234.575
[160]	valid_0's l1: 227.689
[180]	val

[380]	valid_0's l1: 156.029
[400]	valid_0's l1: 155.285
[420]	valid_0's l1: 154.848
[440]	valid_0's l1: 154.187
[460]	valid_0's l1: 153.796
[480]	valid_0's l1: 153.182
[500]	valid_0's l1: 152.912
[520]	valid_0's l1: 152.532
[540]	valid_0's l1: 151.99
[560]	valid_0's l1: 151.661
Early stopping, best iteration is:
[564]	valid_0's l1: 151.54
------------------------------------------------------------
Case :  129
boosting parameter :  goss
Learning_rate :  0.1
max_depth :  5
num_leaves :  15
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001561 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 84
[LightGBM] [Info] Number of data points in the train set: 306768, number of used features: 5
[LightGBM] [Info] Using GOSS
[LightGBM] [Info] Start training from score 934.864036
Training until validation scores don't improve for 10 rounds
[20]	va

[360]	valid_0's l1: 153.93
[380]	valid_0's l1: 153.092
[400]	valid_0's l1: 152.635
[420]	valid_0's l1: 152.167
Early stopping, best iteration is:
[417]	valid_0's l1: 152.071
------------------------------------------------------------
Case :  135
boosting parameter :  goss
Learning_rate :  0.1
max_depth :  15
num_leaves :  15
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001511 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 84
[LightGBM] [Info] Number of data points in the train set: 306768, number of used features: 5
[LightGBM] [Info] Using GOSS
[LightGBM] [Info] Start training from score 934.864036
Training until validation scores don't improve for 10 rounds
[20]	valid_0's l1: 287.055
[40]	valid_0's l1: 197.334
[60]	valid_0's l1: 172.815
[80]	valid_0's l1: 165.205
[100]	valid_0's l1: 161.52
[120]	valid_0's l1: 159.04
[140]	valid

In [55]:
MAE_grid_search

,case number,boosting,Lr,max_depth,num_leaves,MAE
0,1.0,gbdt,0.001,5.0,5.0,537.056790
1,2.0,gbdt,0.001,5.0,10.0,458.751083
2,3.0,gbdt,0.001,5.0,15.0,414.755716
3,4.0,gbdt,0.001,10.0,5.0,537.056790
4,5.0,gbdt,0.001,10.0,10.0,458.785211
...,...,...,...,...,...,...
130,131.0,goss,0.100,10.0,10.0,152.071347
131,132.0,goss,0.100,10.0,15.0,150.491967
132,133.0,goss,0.100,15.0,5.0,164.729394
133,134.0,goss,0.100,15.0,10.0,152.071347


In [65]:
pd.set_option('display.max_rows',135)
MAE_grid_search_rank = MAE_grid_search.sort_values(MAE_grid_search.columns[-1],ascending = True)
MAE_grid_search_rank

,case number,boosting,Lr,max_depth,num_leaves,MAE
35,36.0,gbdt,0.050,15.0,15.0,147.704784
41,42.0,gbdt,0.100,10.0,15.0,148.149539
44,45.0,gbdt,0.100,15.0,15.0,149.402471
134,135.0,goss,0.100,15.0,15.0,149.605157
122,123.0,goss,0.050,10.0,15.0,149.696827
40,41.0,gbdt,0.100,10.0,10.0,150.044331
43,44.0,gbdt,0.100,15.0,10.0,150.044331
125,126.0,goss,0.050,15.0,15.0,150.175329
131,132.0,goss,0.100,10.0,15.0,150.491967
121,122.0,goss,0.050,10.0,10.0,151.468401


## 최고성능 재학습

In [67]:
d_train = lgb.Dataset(train_x, train_y)
d_val = lgb.Dataset(val_x, val_y)

params = {
    'objective': 'regression',
    'metric':'mae',
    'seed':42 ,
    'boosting' : MAE_grid_search_rank.iloc[0,1],
    'learning_rate' : MAE_grid_search_rank.iloc[0,2],
    'max_depth' : int(MAE_grid_search_rank.iloc[0,3]),
    'num_leaves' : int(MAE_grid_search_rank.iloc[0,4])
}

model = lgb.train(params, d_train, 3000, d_val, verbose_eval=20, early_stopping_rounds=10)


[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001187 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 84
[LightGBM] [Info] Number of data points in the train set: 306768, number of used features: 5
[LightGBM] [Info] Start training from score 934.864036
Training until validation scores don't improve for 10 rounds
[20]	valid_0's l1: 403.537
[40]	valid_0's l1: 288.135
[60]	valid_0's l1: 230.448
[80]	valid_0's l1: 198.226
[100]	valid_0's l1: 182.289
[120]	valid_0's l1: 172.962
[140]	valid_0's l1: 167.782
[160]	valid_0's l1: 164.347
[180]	valid_0's l1: 161.96
[200]	valid_0's l1: 160.2
[220]	valid_0's l1: 159.153
[240]	valid_0's l1: 157.994
[260]	valid_0's l1: 157.21
[280]	valid_0's l1: 156.357
[300]	valid_0's l1: 155.635
[320]	valid_0's l1: 155.23
[340]	valid_0's l1: 154.689
[360]	valid_0's l1: 154.284
[380]	valid_0's l1: 153.733
[400]	v

# 추론 및 결과 제출

In [68]:
test = pd.read_csv('data/test.csv')
submission = pd.read_csv('data/sample_submission.csv')

In [69]:
test.head()

,일자|시간|구분
0,2019-01-01 01 A
1,2019-01-01 02 A
2,2019-01-01 03 A
3,2019-01-01 04 A
4,2019-01-01 05 A


In [70]:
submission.head()

,일자|시간|구분,공급량
0,2019-01-01 01 A,0
1,2019-01-01 02 A,0
2,2019-01-01 03 A,0
3,2019-01-01 04 A,0
4,2019-01-01 05 A,0


In [71]:
test['일자'] = test['일자|시간|구분'].str.split(' ').str[0]
test['시간'] = test['일자|시간|구분'].str.split(' ').str[1].astype(int)
test['구분'] = test['일자|시간|구분'].str.split(' ').str[2]

In [72]:
test['일자'] = pd.to_datetime(test['일자'])
test['year'] = test['일자'].dt.year
test['month'] = test['일자'].dt.month
test['day'] = test['일자'].dt.day
test['weekday'] = test['일자'].dt.weekday

In [73]:
test['구분'] = test['구분'].map(d_map)

In [74]:
test_x = test[features]

In [75]:
test_x

,구분,month,day,weekday,시간
0,0,1,1,1,1
1,0,1,1,1,2
2,0,1,1,1,3
3,0,1,1,1,4
4,0,1,1,1,5
...,...,...,...,...,...
15115,6,3,31,6,20
15116,6,3,31,6,21
15117,6,3,31,6,22
15118,6,3,31,6,23


In [76]:
preds = model.predict(test_x)

In [77]:
submission['공급량'] = preds

In [78]:
submission

,일자|시간|구분,공급량
0,2019-01-01 01 A,2162.578630
1,2019-01-01 02 A,1972.627825
2,2019-01-01 03 A,1851.811462
3,2019-01-01 04 A,1897.705573
4,2019-01-01 05 A,2061.499260
...,...,...
15115,2019-03-31 20 H,393.660227
15116,2019-03-31 21 H,388.241693
15117,2019-03-31 22 H,373.396772
15118,2019-03-31 23 H,343.524613


In [80]:
submission.to_csv('./prediction results/grid_search_1.0.csv', index=False)